In [1]:
from datasets import Dataset

# bible_data = Dataset.from_csv("berean_study_bible.csv")
# bible_data = bible_data.rename_columns({
#     "Verse": "verse",
#     "Berean Standard Bible": "text"
#     })
# bible_data = bible_data.filter(lambda x: isinstance(x["text"], str))

/Users/thomaswinterburn/Library/Caches/pypoetry/virtualenvs/wonder-comp-sci-basics-hGs6WSs2-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModel

model_cpkt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_cpkt)
model = AutoModel.from_pretrained(model_cpkt)

/Users/thomaswinterburn/Library/Caches/pypoetry/virtualenvs/wonder-comp-sci-basics-hGs6WSs2-py3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/thomaswinterburn/Library/Caches/pypoetry/virtualenvs/wonder-comp-sci-basics-hGs6WSs2-py3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/thomaswinterburn/Library/Caches/pypoetry/virtualenvs/wonder-comp-sci-basics-hGs6WSs2-py3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._regis

In [3]:
import torch

device = torch.device('cpu')
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [4]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [5]:
# embedding = get_embeddings(bible_data["text"][0])
# embedding.shape

In [6]:
# embeddings_dataset = bible_data.map(
#     lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
# )

In [7]:
# embeddings_dataset.save_to_disk("embedding_dataset")

In [8]:
embeddings_dataset = Dataset.load_from_disk("embedding_dataset")

In [9]:
embeddings_dataset.add_faiss_index(column="embeddings")

100%|██████████| 32/32 [00:00<00:00, 113.34it/s]


Dataset({
    features: ['verse', 'text', 'embeddings'],
    num_rows: 31086
})

In [10]:
question = "Who should we love?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [11]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

: 

In [11]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)


NameError: name 'samples' is not defined

In [12]:
for _, row in samples_df.iterrows():
    print(f"VERSE: {row.verse}")
    print(f"SCORE: {row.scores}")
    print(f"TEXT: {row.text}")
    print("=" * 50)
    print()

NameError: name 'samples_df' is not defined